In [34]:
# establish connection
from os import getenv
from dotenv import load_dotenv
import pyodbc
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=localhost\\DOKUSSERVER;"
                      "Database=Lernperiode12;"
                      "Trusted_Connection=yes;")

cursor = cnxn.cursor()

In [35]:
cursor.execute("SELECT MAX(timestamp) FROM WeatherMeasurement")
last_saved_time = cursor.fetchone()[0]

In [36]:
from datetime import datetime, timedelta
if last_saved_time is None:
    last_saved_time = datetime.now() - timedelta(hours=48)
start = last_saved_time - timedelta(hours=48)
end = datetime.now()

start_str = start.date().isoformat()
end_str   = end.date().isoformat()

In [46]:
# request
import requests
import json
from datetime import datetime

def get_weather_data(latitude, longitude, start, end):
    base_url = "https://api.open-meteo.com/v1/forecast"
    
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": [
            "temperature_2m",
            "relative_humidity_2m",
            "wind_speed_10m",
            "precipitation"
        ],
        "past_days": 2,
        "forecast_days": 0,
        "timezone": "auto"
    }


    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print("Fetch failed:", e)
        return None


# Coordinates for Baden, Switzerland
baden_lat = 47.475361
baden_lon = 8.306372

weather = get_weather_data(baden_lat, baden_lon, start_str, end_str)
print(weather)

{'latitude': 47.48, 'longitude': 8.299999, 'generationtime_ms': 0.10609626770019531, 'utc_offset_seconds': 3600, 'timezone': 'Europe/Zurich', 'timezone_abbreviation': 'GMT+1', 'elevation': 394.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C', 'relative_humidity_2m': '%', 'wind_speed_10m': 'km/h', 'precipitation': 'mm'}, 'hourly': {'time': ['2025-11-26T00:00', '2025-11-26T01:00', '2025-11-26T02:00', '2025-11-26T03:00', '2025-11-26T04:00', '2025-11-26T05:00', '2025-11-26T06:00', '2025-11-26T07:00', '2025-11-26T08:00', '2025-11-26T09:00', '2025-11-26T10:00', '2025-11-26T11:00', '2025-11-26T12:00', '2025-11-26T13:00', '2025-11-26T14:00', '2025-11-26T15:00', '2025-11-26T16:00', '2025-11-26T17:00', '2025-11-26T18:00', '2025-11-26T19:00', '2025-11-26T20:00', '2025-11-26T21:00', '2025-11-26T22:00', '2025-11-26T23:00', '2025-11-27T00:00', '2025-11-27T01:00', '2025-11-27T02:00', '2025-11-27T03:00', '2025-11-27T04:00', '2025-11-27T05:00', '2025-11-27T06:00', '2025-11-27T07:00', '2025

In [49]:
# getting only new rows with pandas
import pandas as pd
import pytz
df = pd.DataFrame(weather["hourly"])
tz = df["time"].dt.tz  # extract tz from pandas series
last_saved_time_tz = last_saved_time.replace(tzinfo=tz)
df_new = df[df["time"] > last_saved_time_tz]

AttributeError: Can only use .dt accessor with datetimelike values

In [45]:
df_new.tail(5)

,time,temperature_2m,relative_humidity_2m,wind_speed_10m,precipitation,cloud_cover


In [39]:
# weather data template
SQL_API_STATEMENT = """
INSERT INTO WeatherMeasurement (
    timestamp,
    temperature,
    humidity,
    windspeed,
    precipitation
) VALUES
( ?, ?, ?, ?, ?)
"""

In [40]:
# insert current weather data
from datetime import datetime

for _, row in df_new.iterrows():
    cursor.execute(
        SQL_API_STATEMENT,
        (
            row["time"],
            row["temperature_2m"],
            row["relative_humidity_2m"],
            row["wind_speed_10m"],
            row["precipitation"]
        )
    )
cnxn.commit()


In [ ]:
# check last data time
# get data from 48 hours
    # define end and start
# compare pulled data with last inserted time
# leave only the data that is higher than max time from db
# insert
# if there is a gap
# then inform there's missing data because pulled from only 48h
# missing: x hours